# Problem Description

This notebook will guide you through implementation of **multivariate linear regression** to to solve the **polynomial regression** problem:

$$
h_{\theta}(x) = \theta_0 + \theta_1 x + \theta_2 x^2 +  \theta_3 x^3 + \theta_4 x^4
= \boldsymbol{\theta}^T \mathbf{x}
$$

$$
\boldsymbol{\theta} = \begin{bmatrix}\theta_0 \\ \theta_1 \\ \theta_2 \\ \theta_3 \\ \theta_4\end{bmatrix}, 
\qquad
\mathbf{x} = \begin{bmatrix}1 \\ x \\ x^2 \\ x^3 \\ x^4\end{bmatrix}
$$

Below, you will follow steps to 

1. Implement the cost function for multivarate linear regression
1. Compare vectorized code with for-loops
1. Implement the normal equations method to solve a multivariate linear regression problem
1. Implement gradient descent for multivariate linear regression
1. Experiment with feature normalization to improve the convergence of gradient descent


## Imports
Run this code.

In [2]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

## Helper functions
Run this code to set up the helper functions. The function ``feature_expansion`` accepts an vector of $n$ scalar x values and returns an $n \times 5$ data matrix by applying the feature expansion $x \mapsto [1, x, x^2, x^3, x^4]$ to each scalar $x$ value.

In [ ]:
def feature_expansion(x, deg):
    if x.ndim > 1:
        raise ValueError('x should be a 1-dimensional array')
    m = x.shape
    x_powers = [x**k for k in range(0,deg+1)]
    X = np.stack( x_powers, axis=1 )

    return X

def plot_model(X_test, theta):
    '''
    Note: uses globals x, y, x_test, which are assigned below
    when the dataset is created. Don't overwrite these variables.
    '''
    y_test = np.dot(X_test, theta)
    plt.scatter(x, y)
    plt.plot(x_test, y_test)
    plt.legend(['Test', 'Train'])

## (2 points) List comprehensions

Read about Python list comprehensions. Explain what is happening in the following line of code

```python
x_powers = [x**k for k in range(0,deg+1)]
```

** *Your answer here* **

## Create a data set for polynomial regression

Read and run the code below. This generates data from a fourth-degree polynomial and then uses feature expansion to set up the problem of learning the polynomial as multivariate linear regression

In [ ]:
# Set random seed
np.random.seed(0)

# Create random set of m training x values between -5 and 5
m = 100
x = np.random.rand(m)*10 - 5   

# Create evenly spaced test x values (for plotting)
x_test  = np.linspace(-5, 5, 100)
m_test  = len(x_test);

# Feature expansion for training and test x values
deg = 4
X      = feature_expansion(x, deg)
X_test = feature_expansion(x_test, deg)

n = deg + 1   # total number of features including the '1' feature

# Define parameters (theta) and generate y values
theta = 0.1*np.array([1, 1, 10, 0.5, -0.5]);
y = np.dot(X, theta) + np.random.randn(m)   # polynomial plus noise

# Plot the training data
plt.scatter(x, y)
plt.title('Training Data')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


In [ ]:
#look at the feature expansion for a single training example
print(x[0]) #original data 
print(X[0]) #data with feature expansion

## Implement the cost function
Follow the instructions to implement the following cost function for multivariate linear regression:

$$J(\theta) = \frac{1}{2}\sum_{i=1}^n(h_{\theta}(x^{(i)})-y^{(i)})^2$$

### (3 points)  Cost function with loops 
First, implement the cost function using a for-loops: `cost_function_loops`.

In [ ]:
def cost_function_loops(X, y, theta):  
    '''
    Compute the cost function for a particular data set and 
    hypothesis (parameter vector)
    
    
    Inputs: 
        X       m x n data matrix
        y       training output (length m vector)
        theta   parameters (length n vector)
    Output:
        cost    the value of the cost function (scalar)
    '''
    # TODO: write correct code to implement the cost function given above
    # Use for-loops for this function
    cost = 0
    return cost

### (3 points)  Vectorized cost function 
Now, implment the same cost function but now WITHOUT any for-loops. You should be using NumPy. This is a "vectorized" version of the cost function: `cost_function_vec`

In [ ]:
def cost_function_vec(X, y, theta):  
    '''
    No for-loops allowed! 
    
    Compute the cost function for a particular data set and 
    hypothesis (parameter vector)
    
    Inputs: 
        X       m x n data matrix
        y       training output (length m vector)
        theta   parameters (length n vector)
    Output:
        cost    the value of the cost function (scalar)
    '''
    
    # TODO: write correct code to implement the cost function given above
    # You CANNOT use for-loops here!
    cost = 0
    return cost

## Test the cost function
Run this to test your cost function.

In [ ]:
np.random.seed(1)

theta_random = np.random.rand(n)
theta_zeros  = np.zeros(n)
theta_ones   = np.ones(n)

print("cost_function_loops")
print("=="*10)
print( "Cost (random): %.2f" % cost_function_loops(X, y, theta_random))  # prints 54523.64
print( "Cost (zeros): %.2f" % cost_function_loops(X, y, theta_zeros))   # prints 845.65
print( "Cost (ones): %.2f" % cost_function_loops(X, y, theta_ones))    # prints 2524681.08
print()
print("cost_function_vec")
print("=="*10)
print( "Cost (random): %.2f" % cost_function_vec(X, y, theta_random))  # prints 54523.64
print( "Cost (zeros): %.2f" % cost_function_vec(X, y, theta_zeros))   # prints 845.65
print( "Cost (ones): %.2f" % cost_function_vec(X, y, theta_ones))    # prints 2524681.08
print()
#Note: The for-loop and vectorized cost function implementations should return the EXACT 
# same results. 

## (2 points)  Time cost functions 

- Run `cost_function_loops` and `cost_function_vec` each 100 times for `theta_random` and the `X` and `y` given above
- Print out the mean and standard deviation across all runs for each function 
- Report which function is faster.  

In [ ]:
np.random.seed(1)
theta_random = np.random.rand(n)
##################
# TODO: implement your code here #
##################

** *Write answer here: report which function is faster* **

### For the rest of this notebook, we will use the vectorized implementation of the cost function 
Run the code below. 

In [ ]:
cost_function = cost_function_vec

In [ ]:
#make sure it works 
cost_function(X, y, theta_random)

## (7 points) Implement first training algorithm: normal equations
Implement a *vectorized* version of the normal equations! If you use for-loops you will not get full credit. 

In [ ]:
def normal_equations(X, y):
    '''
    Train a linear regression model using the normal equations

    Inputs: 
        X       m x n data matrix
        y       training output (length m vector)
    Output:
        theta   parameters (length n vector)

    '''
    # TODO: write correct code to find theta using the normal equations
    m, n = X.shape
    theta = np.zeros(n)
    return theta

## Use normal equations to fit the model
Run this code to test your implementation of the normal equations. If it runs properly you will see a curve that fits the data well. Note the value of the cost function for ``theta_normal_equations``.

In [ ]:
theta_normal_equations = normal_equations(X, y)
plot_model(X_test, theta_normal_equations)
print ("Cost function: %.2f" % cost_function(X, y, theta_normal_equations))

## (7 points) Implement second training algorithm: (vectorized) gradient descent

Implement gradient descent for multivariate linear regression. Make sure your solution is vectorized. If you use for-loops to compute the gradient you will not receive full credit. 

In [ ]:
def gradient_descent( X, y, alpha, iters, theta=None ):
    '''
    Train a linear regression model by gradient descent

    Inputs: 
        X       m x n data matrix
        y       training output (length m vector)
        alpha   step size
        iters   number of iterations
        theta   initial parameter values (length n vector; optional)
    
    Output:
        theta      learned parameters (length n vector)
        J_history  trace of cost function value in each iteration

    '''

    m,n = X.shape
    
    if theta is None:
        theta = np.zeros(n)
    
    # For recording cost function value during gradient descent
    J_history = np.zeros(iters)

    for i in range(0, iters):
        
        # TODO: compute gradient (vectorized) and update theta
        
        # Record cost function
        J_history[i] = cost_function(X, y, theta)
        
    return theta, J_history

## (6 points) Use gradient descent to train the model
* Write code to call your ``gradient_descent`` method to learn parameter
* Plot the model fit (use ``plot_model``)
* Plot the cost function vs. iteration to help assess convergence
* Print the final value of the cost function
* Experiment with different step sizes and numbers of iterations until you can find a good hypothesis. Try to match the cost function value from ``normal_equations`` to two decimal places. How many iterations does this take?

In [ ]:
# TODO: write code

** *Write your answers here* **

## (10 points) Gradient descent with feature normalization
You should have observed
that it takes many iterations of gradient descent to match the cost
function value achieved by the normal equations. Now
you will implement feature normalization to improve the convergence
of gradient descent. Remember that the formula
for feature normalization is:

$$x^{(i)}_j \leftarrow \frac{x^{(i)}_j - \mu_j}{\sigma_j}$$

Here are some guidelines for the implementation:

* The same transformation should be applied to train and test data.

* The values $\mu_j$ and $\sigma_j$ are the mean and standard deviation of
the $j$th column (i.e., feature) in the **training data**. (Hint:
there are numpy functions to compute these.)

* Do not normalize the column of all ones. (Optional question: why?)

* Use broadcasting to do the normalization--don't write for loops

After normalizing both the training data and test data, follow the same steps as above to experiment with gradient descent using the *normalized* training and test data: print the value of the cost function, and create the same plots. Tune the step size and number of iterations again to make gradient descent converge as quickly as possible. How many iterations does it take to match the cost function value from ``normal_equations`` to two decimal places?  


In [ ]:
# TODO: your code for gradient descent with feature normalization

** *Write answer here: how many iterations?* **